# Q6

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import CountVectorizer 
train_data = open('20news-bydate/matlab/train.data') 
test_data = open('20news-bydate/matlab/test.data')
df_data = pd.read_csv(train_data, delimiter=' ', names=['docIndex', 'wordIndex', 'count']) 
df_wordIndex_cnt=df_data.groupby(by=df_data['wordIndex'], as_index=False).agg({'count': 'sum'}) 
df_wordIndex_cnt

,wordIndex,count
0,1,317
1,2,1364
2,3,300
3,4,226
4,5,327
...,...,...
53970,53971,1
53971,53972,1
53972,53973,1
53973,53974,1


In [2]:
# segregating most common words
df_most_common_words = df_wordIndex_cnt[df_wordIndex_cnt['count'] > 1000] 
df_most_common_words

,wordIndex,count
1,2,1364
5,6,1832
8,9,1002
11,12,64170
15,16,11109
...,...,...
4322,4323,1119
5265,5266,1009
7259,7260,1246
10267,10268,1693


### As mentioned in the question we got close to 300 words that appear more that 1000 times 

In [3]:
from sklearn import metrics 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.naive_bayes import BernoulliNB 

In [5]:
termIndices = df_most_common_words.wordIndex.values 
vocab=[] 
with open('20news-bydate/matlab/vocabulary.txt') as f: 
    for x in f: 
        vocab.append(x[:-1]) 
myVocab=[vocab[i-1] for i in termIndices] 
train_set = fetch_20newsgroups(subset='train') 
sparse = CountVectorizer(vocabulary=myVocab) 
train_data = sparse.fit_transform(train_set.data) 
test_set = fetch_20newsgroups(subset='test') 
test_data = sparse.transform(test_set.data) 

## 6(a) Bernoulli Naive Bayes

In [22]:
sparse = CountVectorizer(vocabulary=myVocab,binary=True) 
train = sparse.fit_transform(train_set.data) 
test_set = fetch_20newsgroups(subset='test',shuffle=False) 
test = sparse.transform(test_set.data) 
bernoulli_classifier = BernoulliNB(alpha=0.01) 
bernoulli_classifier.fit(train, train_set.target) 
predictions = bernoulli_classifier.predict(test) 
print("Test accuracy obtained : ",round(100*metrics.accuracy_score(test_set.target, predictions),2),"%")

Test accuracy obtained :  59.58 %


## 6(b) Multinomial Naive Bayes

In [26]:
multinomial_classifier = MultinomialNB(alpha=0.01) 
multinomial_classifier.fit(train, train_set.target) 
predictions = multinomial_classifier.predict(test) 
print("Test accuracy obtained :",round(100*metrics.accuracy_score(test_set.target, predictions),2),"%")

Test accuracy obtained : 71.29 %


## 6(c) LDA

In [37]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.multiclass import OneVsRestClassifier 
tfidf = TfidfVectorizer(vocabulary=myVocab) 
train = tfidf.fit_transform(train_set.data) 
test_set = fetch_20newsgroups(subset='test') 
test = tfidf.transform(test_set.data) 
model = LinearDiscriminantAnalysis(solver='svd') 
model.fit(train.toarray(),train_set.target) 
predictions =model.predict(test.toarray()) 
print("Test accuracy obtained :",round(100*metrics.accuracy_score(test_set.target, predictions),2),"%")

Test accuracy obtained : 73.55 %


## 6(d) Least squares classifier

In [36]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression(C=1.0, penalty='l1', dual=False, solver='liblinear')

model = OneVsRestClassifier(model) 
model.fit(train,train_set.target) 
predictions =model.predict(test.toarray()) 
print("Test accuracy obtained :",round(100*metrics.accuracy_score(test_set.target, predictions),2),"%")

Test accuracy obtained : 77.18 %
